# Домашнее задание к лекции "Понятие класса"
Преподаватель: **Константин Башевой**

09.04.2020

## Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах eur и usd будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [5]:
from libs.exchange import Rate

In [37]:
class RateWithMaxAndDiff(Rate):
    """ во избежание почти внешней библиотеки используется наследование класса """
    def __init__(self, format='full', diff=False):
        super().__init__(format)
        self.diff = diff
    
    def currency_max(self):  # ЗАДАНИЕ 1
        """ Получение названия валюты с максимальным значением курса """
        r = self.exchange_rates()

        # формирование списка пар <название валюты>-<текущий курс>
        currency_list = [(r[currency]['Name'], r[currency]['Value']) for currency in r]       
                                 
        # выбор пары с максимальным курсом       
        max_ = max(currency_list, key = lambda x: x[1])  
#         currency_list.sort(key = lambda x: x[1])  # сортивровка списка по значению курса                    
        
        return max_[0]
       
    def make_format_diff(self, currency): # ЗАДАНИЕ 2
        """
        Возвращает информацию о валюте currency в трех вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        - текущий курс 
        Rate('value').make_format('EUR')
        79.4966
        
        - изменение курса
        Rate('value', diff=True).make_format('EUR')
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value':
                if not self.diff:
                    return response[currency]['Value']  # курс
                return response[currency]['Value'] - response[currency]['Previous']  # изменение курса
                    
        
        return 'Error'    
    
    
    def eur(self):
        """Возвращает 
        - курс евро на сегодня в формате self.format
        - или его изменение если format == 'value', diff = True
        """         
        return self.make_format_diff('EUR')
    
    def usd(self): 
        """Возвращает 
        - курс доллар на сегодня в формате self.format
        - или его изменение если format == 'value', diff = True
        """         
        return self.make_format_diff('USD')    

In [40]:
# ЗАДАНИЕ 1
cur = RateWithMaxAndDiff()
print(f'Валюта с максимальным значением курса: {cur.currency_max()}')

Валюта с максимальным значением курса: СДР (специальные права заимствования)


In [41]:
# ЗАДАНИЕ 2
cur2 = RateWithMaxAndDiff(format = 'full', diff=True)
print(cur2.eur())
print(cur2.usd())
cur2.format = 'value'
print(f'Изменение курса евро: {cur2.eur()}')
print(f'Изменение курса доллара: {cur2.usd()}')

{'ID': 'R01239', 'NumCode': '978', 'CharCode': 'EUR', 'Nominal': 1, 'Name': 'Евро', 'Value': 81.0882, 'Previous': 82.2341}
{'ID': 'R01235', 'NumCode': '840', 'CharCode': 'USD', 'Nominal': 1, 'Name': 'Доллар США', 'Value': 74.605, 'Previous': 75.7499}
Изменение курса евро: -1.1458999999999975
Изменение курса доллара: -1.1448999999999927
